In [24]:
def is_Groebner(F):
    """
    Revisa si una lista de polinomios F es una base de Groebner usando el
    Criterio de Buchberger.
    
    INPUT:
    - F: lista de polinomios en un anillo multivariable
    
    OUTPUT:
    - True si F es base de Groebner, False si no lo es
    """
    # Manejar caso de lista vacía o ideal cero
    if not F:
        return True
    
    # Revisar todos los pares (i, j) con i < j
    for i in range(len(F)):
        for j in range(i + 1, len(F)):
            # Saltamos si alguno de los polinomios es cero
            if F[i] == 0 or F[j] == 0:
                continue
                
            # Calcular el S-polinomio
            S_poly = S_polynomial(F[i], F[j])
            
            # Calcular el residuo de la división por F
            # No nos importa el cociente (el _), solo el residuo
            _, remainder = division_algorithm(S_poly, F)
            
            # Si el residuo es distinto de cero, falló el criterio: NO es base de Groebner
            if remainder != 0:
                return False
    
    # Si todos los S-polinomios reducen a cero, entonces sí es base
    return True


def division_algorithm(f, F):
    """
    Implementación del algoritmo de división del §3 (Teorema 3).
    
    INPUT:
    - f: polinomio a dividir
    - F: s-tupla ordenada de polinomios (f_1, ..., f_s)
    
    OUTPUT:
    - q: lista de cocientes [q_1, ..., q_s]
    - r: el residuo (resto)
    """
    # Inicializamos variables 
    s = len(F)
    q = [0] * s  # q_1, ..., q_s
    r = 0
    p = f
    
    while p != 0:
        division_occurred = False # Bandera para saber si dividimos en este paso
        i = 0
        
        while i < s and not division_occurred:
            fi = F[i]
            # Revisar si el término líder de fi divide al de p
            if fi != 0 and fi.lt().divides(p.lt()):
                # Paso de División: LT(f_i) divide a LT(p)
                quotient_term = p.lt() // fi.lt()  # División exacta en el anillo
                q[i] += quotient_term
                p -= quotient_term * fi
                division_occurred = True
            else:
                i += 1
        
        if not division_occurred:
            # Paso del Residuo: el término líder de p va al residuo
            r += p.lt()
            p -= p.lt()
    
    return q, r


def S_polynomial(f, g):
    """
    Calcula el S-polinomio de f y g como se define en la Def 4 del §6.
    
    S(f, g) = (x^gamma / LT(f)) * f - (x^gamma / LT(g)) * g
    donde x^gamma = mcm(LM(f), LM(g))
    """
    R = f.parent()
    
    # Sacamos términos líderes (LT), monomios líderes (LM) y coeficientes líderes (LC)
    LT_f = f.lt()
    LT_g = g.lt()
    LM_f = f.lm()
    LM_g = g.lm()
    LC_f = f.lc()
    LC_g = g.lc()
    
    # Calcular el MCM (gamma) de los monomios líderes
    gamma = lcm_monomials(LM_f, LM_g)
    
    # Calcular las partes monomiales: x^gamma / LM(f) y x^gamma / LM(g)
    monom1 = gamma // LM_f
    monom2 = gamma // LM_g
    
    # Aplicar la fórmula del libro para el S-poly
    term1 = (monom1 * f) / LC_f
    term2 = (monom2 * g) / LC_g
    
    return term1 - term2


def lcm_monomials(m1, m2):
    """
    Calcula el Mínimo Común Múltiplo (LCM) de dos monomios.
    Para x^alpha y x^beta, el LCM es x^gamma donde gamma_i = max(alpha_i, beta_i).
    """
    exponents1 = m1.exponents()[0]
    exponents2 = m2.exponents()[0]
    
    # Tomar el máximo componente a componente como dice el texto
    lcm_exponents = [max(e1, e2) for e1, e2 in zip(exponents1, exponents2)]
    
    return m1.parent().monomial(*lcm_exponents)

In [25]:
R.<x,y,z> = PolynomialRing(QQ, order='lex')

# Caso 1: Una base de Groebner conocida (debería dar True)
F1 = [x + z, y - z]
print("Probando base de Groebner conocida:")
print(is_Groebner(F1))

# Caso 2: No es base de Groebner (debería dar False)
F2 = [x^2, x*y + y^2]
print("Probando base que NO es Groebner:")
print(is_Groebner(F2))

# Probando el algoritmo de división
print("\nProbando algoritmo de división:")
f = x*y^2 + 1
F3 = [x*y + 1, y + 1]
q, r = division_algorithm(f, F3)
print(f"Dividendo (f): {f}")
print(f"Divisores (F): {F3}")
print(f"Cocientes (q): {q}")
print(f"Residuo (r): {r}")

# Probando cálculo del S-polinomio
print("\nProbando S-polinomio:")
f = x*y + 1
g = y + 1
S = S_polynomial(f, g)
print(f"S({f}, {g}) = {S}")

Probando base de Groebner conocida:
True
Probando base que NO es Groebner:
False

Probando algoritmo de división:
Dividendo (f): x*y^2 + 1
Divisores (F): [x*y + 1, y + 1]
Cocientes (q): [y, -1]
Residuo (r): 2

Probando S-polinomio:
S(x*y + 1, y + 1) = -x + 1
